## 타깃을 모르는 비지도 학습

타깃이 없을 때 사용하는 머신 러닝 알고리즘 : 비지도 학습

## 과일 사진 데이터 준비



In [ ]:
!wget https://bit.ly/fruits_300 -O fruits_300.npy # content file path

그 후 파일 데이터 로드, 넘파이와 맷플롯립 패키지 사용

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fruits = np.load('fruits_300.npy')

print(fruits.shape)

print(fruits.shape) 명령어는 fruits의 크기를 나타내는데 (300, 100, 100)으로 리턴한다.
이때 첫번째 300은 300개의 데이터, 두번째와 세번째 100은 100x100 사이즈를 가지고 있다는 것을 뜻한다.


In [ ]:
print(fruits[0,0,:])

In [ ]:
plt.imshow(fruits[0], cmap='gray')
plt.show()

해당 명령어 실행 시 바탕이 검은 색으로 표시. 0의 값을 가지는 쓰레기값.
넘파이로 변환 시 반전시킨 것. cmap 조건을 'gray_r'로 하면 원래의 사진으로 나온다.

In [ ]:
fig, axs = plt.subplots(1,2)
axs[0].imshow(fruits[100], cmap='gray_r')
axs[1].imshow(fruits[200], cmap='gray_r')
plt.show()

이 데이터는 사과, 파인애플, 바나나가 각각 100개씩 사진이 들어있다.

맷플롯립의 subplots() 함수는 여러개의 그래프를 배열처럼 쌓을 수 있도록 도와준다. subplots(1,2)를 통해 1행과 2열을 지정

## 픽셀값 분석하기
fruits 데이터를 각각 나눠보고 펼쳐보자.
슬라이싱 연산자를 사용해 나눈다.
100x100인 이차원 배열을 길이가 10000인 1차원 배열로 나눈다.

In [ ]:
apple = fruits[0:100].reshape(-1, 100*100)
pineapple = fruits[101:200].reshape(-1, 100*100)
banana = fruits[201:300].reshape(-1, 100*100)
print(apple.shape)

각 배열마다의 샘플의 픽셀 평균값을 계산, mean()매서드를 사용하여 평균값을 계산, axis = 1로 설정

In [ ]:
print(apple.mean(axis=1))

그다음 히스토그램을 통해 각 평균 값을 한눈에 비교할 수 있도록 나타낸다.


In [ ]:
plt.hist(np.mean(apple, axis = 1), alpha = 0.8)
plt.hist(np.mean(pineapple, axis = 1), alpha = 0.8)
plt.hist(np.mean(banana, axis = 1), alpha = 0.8)
plt.legend(['apple','pineapple','banana'])
plt.show()

히스토그램을 보면 현재 바나나는 40 아래 집중, 사과와 파인애플은 90~100 사이.

픽셀값으로 사과와 파인애플은 구별 가능, 하지만 파인애플과 바나나는 구별할 수 없다.

다음 방법은 샘플의 평균값이 아니라 픽셀 별 평균 값을 비교하는 방법이다.

픽셀 별 평균을 구하는 방법은 axis = 0 으로 설정하면 된다.

In [ ]:
fig, axs = plt.subplots(1,3,figsize = (20, 5))
axs[0].bar(range(10000), np.mean(apple, axis = 0))
axs[1].bar(range(10000), np.mean(pineapple, axis = 0))
axs[2].bar(range(10000), np.mean(banana, axis = 0))
plt.show()

순각 그래프를 보면 과일마다 값이 높은 구간이 다르다. 사과는 아래쪽으로 갈수록 값이 높아지고, 파인애플은 비교적 고르고, 바나나는 중앙의 픽셀값이 높다.

픽셀 평균값을 100x100으로 바꿔 이미지처럼 출력하여 그래프 비교하면 잘 보인다.

In [ ]:
apple_mean = np.mean(apple, axis = 0).reshape(100,100)
pineapple_mean = np.mean(pineapple, axis = 0).reshape(100,100)
banana_mean = np.mean(banana, axis = 0).reshape(100,100)
fig, axs = plt.subplots(1,3, figsize = (20, 5))
axs[0].imshow(apple_mean, cmap = 'gray_r')
axs[1].imshow(pineapple_mean, cmap = 'gray_r')
axs[2].imshow(banana_mean, cmap = 'gray_r')
plt.show()

## 평균값과 가까운 사진 고르기
사과 사진의 평균값인 apple_mean과 가장 가까운 사진을 고름(절대값 오차 사용) fruits 배열에 있는 모든 샘플에서 apple_mean을 뺀 절댓값의 평균을 계산

절댓값은 abs() 함수 사용

그다음 abs_diff는 (300,100,100) 크기의 배열, 각 샘플에 대한 평균을 구하기 위해..

abs_mean은 각 샘플의 오차 평균이므로 1차원 배열

In [ ]:
abs_diff = np.abs(fruits - apple_mean)
abs_mean = np.mean(abs_diff, axis = (1,2))
print(abs_mean.shape)

그다음, 이 값과 가장 작은 순서대로 100개를 골라본다. apple_mean과 오차가 가장 작은 샘플 100개를 고른다는 것이다.
np.argsort() 함수는 작은 것에서 큰 순서대로 나열한 abs_mean 배열의 인덱스 반환.

In [ ]:
apple_index = np.argsort(abs_mean)[:100]
fig, axs = plt.subplots(10, 10, figsize = (10, 10))
for i in range(10):
    for j in range(10):
        axs[i,j].imshow(fruits[apple_index[i*10+j]], cmap='gray_r')
        axs[i,j].axis('off')
plt.show()

먼저 subplot() 함수로 10x10, 100개의 서브그래프를 만들고 전체 그래프의 크기를 figsize = (10,10)으로 설정.

다음 2중 for문으로 10개의 행과 열에 이미지 출력. axs는 10,10 크기의 2차원 배열.

흑백 사진에 있는 픽셀 값을 사용해 과일 사진을 모으는 작업. 비슷한 샘플을 모으는 작업을 '군집'이라고 한다. 군집 알고리즘에서 만든 그룹을 '클러스터'라고 한다.

여기서는 타깃값을 알고 있었기 때문에 사과, 파인애플, 바나나의 사진 평균값을 계산하여 가장 가까운 과일을 찾을 수 있었다. 실제 비지도 학습에서는 타깃값을 모르기 때문에 샘플의 평균값을 미리 구할 수 없다.

타깃값을 모르면서 세 과일의 평균값을 찾기 위해선 k-평균 알고리즘을 사용.

In [ ]:
# 바나나 특정하기
abs_diff = np.abs(fruits - banana_mean)
abs_mean = np.mean(abs_diff, axis = (1,2))

banana_index = np.argsort(abs_mean)[:100] # 배열 정렬 후 인덱스 반환
fig, axs = plt.subplots(10, 10, figsize = (10, 10))
for i in range(10):
    for j in range(10):
        axs[i,j].imshow(fruits[banana_index[i*10+j]], cmap = 'gray_r')
        axs[i,j].axis('off')
plt.show()